In [ ]:
import matplotlib.pyplot as plt
import ROOT 
import numpy as np
import os,copy
import itertools as itrTools

In [ ]:
# kWhite  = 0,   kBlack  = 1,   kGray    = 920,  kRed    = 632,  kGreen  = 416,
# kBlue   = 600, kYellow = 400, kMagenta = 616,  kCyan   = 432,  kOrange = 800,
# kSpring = 820, kTeal   = 840, kAzure   =  860, kViolet = 880,  kPink   = 900

In [ ]:
import ROOT 
import operator 
import array


def getDefaultCanvas():
    
    defaultCanvasParams=dict({"CMS":{},"TAG":{},"EnLumi":{}})
    defaultCanvasParams["CMS"]["text"]         = "CMS"
    defaultCanvasParams["CMS"]["textSize"]     = 0.05
    defaultCanvasParams["CMS"]["textFont"]     = 61   
    defaultCanvasParams["CMS"]["xpos"]         = 0.15   
    defaultCanvasParams["CMS"]["ypos"]         = 0.98+0.004
    
    # for the "preliminary"
    defaultCanvasParams["TAG"]["text"]         = "internal"
    defaultCanvasParams["TAG"]["textSize"]     = 0.76*defaultCanvasParams["CMS"]["textSize"]  
    defaultCanvasParams["TAG"]["textFont"]     = 52   
    defaultCanvasParams["TAG"]["xpos"]         = 0.15  + 0.12
    defaultCanvasParams["TAG"]["ypos"]         = 0.98  - 0.004
    
    # for the "2018A [ 1 fb^{-1}] 13 TeV" label
    defaultCanvasParams["EnLumi"]["text"]       = "               13 TeV"
    defaultCanvasParams["EnLumi"]["textSize"]  = 0.76*defaultCanvasParams["CMS"]["textSize"]  
    defaultCanvasParams["EnLumi"]["textFont"]  = 42   
    defaultCanvasParams["EnLumi"]["xpos"]      = 0.85   
    defaultCanvasParams["EnLumi"]["ypos"]      = 0.95

    return defaultCanvasParams


def getCanvas2018A():
    cPars=getDefaultCanvas()
 #   cPars["EnLumi"]["text"]="   13 TeV"
    return cPars

def getCanvasParams(tag=None):
    cPars=getDefaultCanvas()
    print("Canvas Tag : ", tag )
    if tag=="2018Full":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "SingleEG 2018 (58.9fb^{-1}) 13 TeV"
        return cPars
    if tag=="Run3MC":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3 MC, 13.6 TeV"
        return cPars
    if tag=="Run3_0p900":
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       = "Run 3,900 GeV"
        return cPars
    if tag!=None:
        cPars["EnLumi"]["xpos"]       =  0.961
        cPars["EnLumi"]["text"]       =  tag
          
    return cPars

def getFitParams(hist0):
    mean  = hist0.GetMean()
    stdDev= hist0.GetStdDev()
    minX=mean-2*stdDev
    maxX=mean+2*stdDev
    hist0.Fit("gaus","Q0","",minX,maxX);
    f=hist0.GetFunction("gaus");
    return [f.GetParameter(1),f.GetParameter(2)]

def saveTheDictionary(aCollection,fname=None,folder=None):
    outfile=folder
    if fname!=None and outfile==None:
        outfile=ROOT.TFile(fname,"RECREATE")
    outfile.cd()
    for key in aCollection:
#         print("Doing : ",key)
        if type(aCollection[key])==type({}):
            print("\tMakdig dir : ",key)
            inFolder=outfile.mkdir(key)
            saveTheDictionary(aCollection[key],None,inFolder)
        else:
            aCollection[key].Write()
    if fname!=None:
        outfile.Close()

        
def getTheObjectsFromFile(aFile):
    histStore={}
    for key in aFile.GetListOfKeys():
        kk=key.GetName()
        curObj=aFile.Get(kk)
        if curObj.Class_Name()=='TDirectoryFile':
            histStore[kk]=getTheObjectsFromFile(curObj)
        else:
            histStore[kk]=curObj       
    return histStore

colurWheel=[ 
	   ROOT.kBlack  ,
	   ROOT.kRed    ,
	   ROOT.kGreen  ,
       ROOT.kBlue   ,
	   ROOT.kMagenta ,
	   ROOT.kYellow ,
	   ROOT.kOrange ,
       ROOT.kSpring ,
	   ROOT.kCyan   ,
	   ROOT.kAzure   ,
	   ROOT.kTeal   ,
	   ROOT.kGray    ,
	   ROOT.kViolet ,
	   ROOT.kPink   ,
    ]

class Plot:
    def __init__(self, **args):
        self.name        = args.get("Name", "plot")
        self.legend      = args.get("Legend",None)
        self.histo       = args.get("Histo", None)
        self.fit         = args.get("Fit", None)
        self.markerColor = args.get("MarkerColor", ROOT.kBlue+2)
        self.markerStyle = args.get("MarkerStyle", 22)
        self.markerSize = args.get("MarkerSize", 5)
        self.lineColor   = args.get("LineColor", ROOT.kBlue+2)
        self.lineStyle   = args.get("LineStyle", 1)
        self.lineWidth   = args.get("LineWidth", 1)
        self.options   = args.get("Options", "")
        self.drawLegend   = args.get("DrawLegend",False)
        self.drawLine     = args.get("DrawLine", False)
        self.histo.SetName(self.name+"_histo")
        self.normalize =args.get("Normalize",False)
        self.scaleFactor =args.get("ScaleFactor",None)
        self.doFit =args.get("doFit",False)
        self.isTH1 = args.get("isTH1",True)

class PlotCanvas:
    def __init__(self, **args):
        self.name  = ""
        self.plots = []
        self.rates = []
        self.plotDir =  args.get("prefix","plots/")
        self.yRange = (0.0, 1.0)
        self.xRange = (0, 100)
        self.yRange2 = (0.0, 1.0)
        self.xTitle = args.get("xTitle","E_{T}^{e offl} [GeV]")
        self.yTitle = args.get("yTitle","a.u")
        self.yTitle2 = args.get("yTitle","a.u")
        self.legendPosition = (0.4,0.2,0.9,0.6)
        self.descPosition = (0.6,0.58)
        self.desc = args.get("desc",["L1 Efficiency"])
        self.logx = args.get("logx",False)
        self.logy = args.get("logy",False)
        self.logy2 = args.get("logy2",False)
        self.canvasParams = args.get("canvasPars",getDefaultCanvas())    
        self.setPlotStyle()

    def addRate(self, rate):
        self.rates.append(rate)
    def addPlot(self, plot):
        self.plots.append(plot)
    def clearPlots(self):
        self.plots=[]


    def plot(self):
        xmax=0.0
        xmin=1e9
        xmax_=1e9
        xmin_=0.0
        ymax=0.0
        ymin=1e9
        ymax_=1e9
        ymin_=0.0
        for plot in self.plots:
            histo = plot.histo
            print("Scale Factor  : before  = ",plot.scaleFactor)    
            if plot.normalize:
                if plot.scaleFactor==None:
                    plot.scaleFactor=1.0/histo.Integral()
                else:
                    plot.scaleFactor/=histo.Integral()
                print("Integral = = ",1.0/histo.Integral())    
            if plot.scaleFactor!=None:
                histo.Scale(plot.scaleFactor)
            print("Scaled the histogram by a factor of : ",plot.scaleFactor)
            print("             Integral after Scaling : ",histo.Integral())
            
            if self.yRange[1]=='auto':
                ymax_=histo.GetMaximum()
                if ymax < ymax_*1.1:
                    ymax=ymax_*1.1
            if self.yRange[0]=='auto': 
                ymin_=histo.GetMinimum()
                if ymin > ymin_*0.8:
                    ymin=ymin_*0.8
            
            if self.xRange[1]=='auto':
                xmax_=histo.GetXaxis().GetBinCenter( histo.GetNbinsX() ) + 0.5*histo.GetBinWidth(histo.GetNbinsX())
                print(xmax_,xmax)
                if abs(xmax) < abs(xmax_):
                    xmax=xmax_
            if self.xRange[0]=='auto': 
                xmin_=histo.GetXaxis().GetBinCenter(1) - 0.5*histo.GetBinWidth(1)
                print(xmin_,xmin)
                if abs(xmin) > abs(xmin_):
                    xmin=xmin_
        
        if self.yRange[0]=='auto':
            self.yRange=(ymin,self.yRange[1])
        if self.yRange[1]=='auto':
            self.yRange=(self.yRange[0],ymax)
        if self.xRange[0]=='auto':
            self.xRange=(xmin,self.xRange[1])
        if self.xRange[1]=='auto':
            self.xRange=(self.xRange[0],xmax)

#         print("Setting  X-RANGE  : ",self.xRange)
#         print("Setting  Y-RANGE  : ",self.yRange)
        
#         print("xmax xmin : ",xmin,xmax)
        canvas = ROOT.TCanvas("c_"+self.name, self.name, 700, 800)
        canvas.SetGrid()
        hDummy = ROOT.TH1F("hDummy_"+self.name, self.name, 1, self.xRange[0],self.xRange[1])

        #print("ymin_,ymax_",ymin_,ymax_,"ymin , ymax - > " ,ymin,ymax)
            #print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
        hDummy.SetAxisRange(self.yRange[0], self.yRange[1], "Y")
        hDummy.SetXTitle(self.xTitle)
        hDummy.SetYTitle(self.yTitle)
        hDummy.Draw("C")
        if self.logx : canvas.SetLogx()
        if self.logy : canvas.SetLogy()
        self.logz= True
        if self.logz : canvas.SetLogz()
        
        
        CMSbox=self.getCMSBox()
        extraTextBox=self.getExtraTextBox()
        lumibox=self.getLumiBox()


        selbox=[]
        n=0
        for inx in range(len(self.desc)):
            selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, self.desc[n]))
            selbox[-1].SetNDC()
            selbox[-1].SetTextSize(0.035)
            selbox[-1].SetTextFont(12)
            selbox[-1].SetTextAlign(13)
            n+=1
        for plot in self.plots:
            if(plot.doFit):
                histo = plot.histo
                fitP  = getFitParams(histo)
                strFit='('+'{0:0.3f}'.format(fitP[0])+ ' , '+'{0:0.3f}'.format(fitP[1]) +')'
                selbox.append(ROOT.TLatex  (self.descPosition[0], self.descPosition[1] -n*0.04, strFit))
                selbox[-1].SetNDC()
                selbox[-1].SetTextSize(0.035)
                selbox[-1].SetTextFont(12)
                selbox[-1].SetTextColor(plot.lineColor)
                selbox[-1].SetTextAlign(13)
                n+=1

        #Line legend
        legend = ROOT.TLegend(self.legendPosition[0],self.legendPosition[1],self.legendPosition[2],self.legendPosition[3])
        legend.SetTextFont(42)
        legend.SetFillColor(0)

        for plot in self.plots:
            histo = plot.histo
            print("Ploting hist : ",histo.GetName())
            if(plot.isTH1):
                print( "\tX : Min - > Max  : " ,histo.GetBinCenter(1)," -> ", histo.GetBinCenter(histo.GetNbinsX())," [ ",histo.GetBinCenter(2)-histo.GetBinCenter(1)," ]" )
                print( "\tY : Min - > Max  : " ,histo.GetMinimum()," -> ", histo.GetMaximum())
                
            histo.SetMarkerStyle(plot.markerStyle)
            histo.SetMarkerSize(plot.markerSize)
            histo.SetMarkerColor(plot.markerColor)
            histo.SetLineStyle(plot.lineStyle)
            histo.SetLineWidth(plot.lineWidth)
            histo.SetLineColor(plot.lineColor)
            #histo.Draw("same e")
#             print(plot.options)
            histo.Draw("same "+plot.options)
#             print( histo.GetOption())

            if(plot.legend):
                legend.AddEntry(histo, plot.legend, "pe")
                legend.SetTextSize(0.025) 
            if(plot.drawLegend) :  legend.Draw()
        
        scale = (self.yRange[1] -self.yRange[0])/(self.yRange2[1] - self.yRange2[0])
        y20=self.yRange2[0]

        if(self.logy2):
            scale = (self.yRange[1] -self.yRange[0])/(ROOT.log(self.yRange2[1]) - ROOT.log(self.yRange2[0]) )
            y20 = ROOT.log(self.yRange2[0])


        #####   begin :            For plotting a graph in the Second Axis         #####

        if len(self.rates) >0 :
            axis=None
            canvas.SetRightMargin(0.18);
            if(self.logy2):
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+LG")
            else :
                axis = ROOT.TGaxis(self.xRange[1],self.yRange[0],self.xRange[1], self.yRange[1],self.yRange2[0],self.yRange2[1],510,"+L")
            axis.SetLineColor(ROOT.kRed);
            axis.SetTextColor(ROOT.kRed);
            axis.SetTitle(self.yTitle2)
            axis.SetTitleColor(ROOT.kBlack)
            axis.SetTitleOffset(1.2)
            axis.Draw();

        for rate in self.rates:
            histo = rate.histo
            if self.logy2:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = ( ROOT.log(content + 1e-20 )  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
                    #print("\t ",content," -> ",scaledContent,"  = ( ",ROOT.log(content + 1e-20),"  - ",y20," )* ",scale)
            else:
                for i in range(histo.GetNbinsX()):
                    content=histo.GetBinContent(i+1)
                    scaledContent = (content  - y20 )* scale
                    histo.SetBinContent(i+1,scaledContent)
            
            histo.SetMarkerStyle(rate.markerStyle)
            histo.SetMarkerColor(rate.markerColor)
            histo.SetLineColor(rate.markerColor)
            histo.Draw("pl same "+ rate.options)
            legend.AddEntry(histo, rate.legend, "pe")
            legend.SetTextSize(0.025) 
            if(rate.drawLegend) :  legend.Draw()
        
        #####   end :            For plotting a graph in the Second Axis         #####
            
        CMSbox.Draw()
        extraTextBox.Draw()
        lumibox.Draw()
        for selb in selbox:
            selb.Draw()
        
        canvas.Print(self.plotDir+"/"+self.name+".pdf", "pdf")
        canvas.Print(self.plotDir+"/"+self.name+".png", "png")

        return canvas

    def getCMSBox(self):
        par=self.canvasParams["CMS"]
        CMSbox       = ROOT.TLatex  (par["xpos"],par["ypos"], par["text"])
        CMSbox.SetNDC()
        CMSbox.SetTextSize(par["textSize"])
        CMSbox.SetTextFont(par["textFont"])
        CMSbox.SetTextColor(ROOT.kBlack)
        CMSbox.SetTextAlign(13) 
        return CMSbox
   
    def getExtraTextBox(self):
        par=self.canvasParams["TAG"]
        extraTextBox = ROOT.TLatex  (par["xpos"], par["ypos"], par["text"])
        extraTextBox.SetNDC()
        extraTextBox.SetTextSize(par["textSize"])
        extraTextBox.SetTextFont(par["textFont"])
        extraTextBox.SetTextColor(ROOT.kBlack)
        extraTextBox.SetTextAlign(13)
        
        return extraTextBox

    def getLumiBox(self):
        par=self.canvasParams["EnLumi"]
        lumibox = ROOT.TLatex  ( par["xpos"], par["ypos"], par["text"])
        lumibox.SetNDC()
        lumibox.SetTextAlign(31)
        lumibox.SetTextSize(par["textSize"])
        lumibox.SetTextFont(par["textFont"])
        lumibox.SetTextColor(ROOT.kBlack)
        
        return lumibox
        
    def setPlotStyle(self):
        ROOT.gROOT.SetStyle("Plain")
        ROOT.gStyle.SetOptStat()
        ROOT.gStyle.SetOptFit(0)
        ROOT.gStyle.SetOptTitle(0)
        ROOT.gStyle.SetFrameLineWidth(2)
        ROOT.gStyle.SetPadBottomMargin(0.13)
        ROOT.gStyle.SetPadLeftMargin(0.15)
        ROOT.gStyle.SetPadTopMargin(0.06)
        ROOT.gStyle.SetPadRightMargin(0.05)

        ROOT.gStyle.SetLabelFont(42,"X")
        ROOT.gStyle.SetLabelFont(42,"Y")
        ROOT.gStyle.SetLabelSize(0.04,"X")
        ROOT.gStyle.SetLabelSize(0.04,"Y")
        ROOT.gStyle.SetLabelOffset(0.01,"Y")
        ROOT.gStyle.SetTickLength(0.02,"X")
        ROOT.gStyle.SetTickLength(0.02,"Y")
        ROOT.gStyle.SetLineWidth(2)
        ROOT.gStyle.SetTickLength(0.02 ,"Z")

        ROOT.gStyle.SetTitleSize(0.1)
        ROOT.gStyle.SetTitleFont(42,"X")
        ROOT.gStyle.SetTitleFont(42,"Y")
        ROOT.gStyle.SetTitleSize(0.05,"X")
        ROOT.gStyle.SetTitleSize(0.05,"Y")
        ROOT.gStyle.SetTitleOffset(1.1,"X")
        ROOT.gStyle.SetTitleOffset(1.4,"Y")
        ROOT.gStyle.SetOptStat(0)
        ROOT.gStyle.SetPalette(1)
        ROOT.gStyle.SetPaintTextFormat("3.2f")
        ROOT.gROOT.ForceStyle()

col=[ 0,1,2,ROOT.kGreen+1,4,6,ROOT.kOrange+2,ROOT.kMagenta+2,9,11,13,14,15  ]

def getDefaultPlotParams(col=1,marker=22,markerSize=1):
    return {'Legend':"Data : 2018, Unpacked",
                          'MarkerColor' : col,
                          'LineColor':col,
                          'LineWidth':2,
                          'LineStyle':9,
                          'MarkerStyle':marker,
                          'MarkerSize':markerSize,
                          'DrawLegend':True,
                          'Option':'pc',
                           'isMC' : True
            } 

def getDefaultPlot(prefix='plots/',name='defaultName',cPars=getCanvasParams('GEN')):
#     plot=putil.PlotCanvas(prefix=prefix,canvasPars=cPars)
    plot=PlotCanvas(prefix=prefix,canvasPars=cPars)
    
    plot.name   = name
    plot.xRange = (3.0,50.0)  
    plot.yRange = (0.0,1.02) 
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offl.}"
    plot.desc   = ["L1 EG Isolation"] 
    plot.legendPosition = (0.6,0.5,0.90,0.65)
    plot.descPosition   = (0.6,0.75)     
    plot.logx = False
    plot.logy = False
    return plot

### GenPlots

In [ ]:
! mkdir results/plots -p

pltPrefix='results/plots/gridV4_WithPFA1p_v2prime/SingleEG'

if not os.path.exists(pltPrefix):
    os.system("mkdir -p "+pltPrefix)

In [ ]:
fileList={
   'gridA' : 'results/isoDevV4_withPFA1pGridA/SingleEG/FixedRateScans_sweta/16.7kHz/allOptions.root',
#    'gridC' : 'results/isoDevV4sigmoid/SingleEG/FixedRateScans/16.7kHz/allOptions.root',
   'baseline' : 'results/isoDevV4_withPFA1pGridA/SingleEG/FixedRateScans_sweta/16.7kHz/selectedOptions.root',
}

histStore={}
fileStore={}
for tag in fileList:
    print('Tag : ',tag)
    fileStore[tag]=ROOT.TFile(fileList[tag],'READ')
    histStore[tag]=getTheObjectsFromFile(fileStore[tag])

In [ ]:
for tag in histStore['baseline']['baselineDir']:
    if 'se' in tag:
        print(tag)
#     continue
    print(tag)

In [ ]:
baseline_eff=histStore['baseline']['baselineDir']['divide_pT_pass_TightIsoEG34_by_pT_all']
baseline_rate=histStore['baseline']['baselineDir']['SingleEG_rate_TightIso']

In [ ]:
for tag in histStore['gridA']['100']:
    print(tag)
#     for i in genHistStore['c3_1_c4_1'][tag]:
#         print("\t",i)

## Rates

In [ ]:
if False:  # Rates All Grid Best option
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='BestPoints_rates',cPars=cPars) 
    plot.legendPosition = (0.55,0.55,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['Linear Relaxation']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    for tag in ['gridC']:
        for opt in ['264']:
            for hName in histStore[tag][opt]:
                if 'rate' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='#sigma-oid Option : '+opt
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['MarkerStyle']= 22
                pparams['LineSize'] = 2
                pparams['Options']='pec'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    for tag in ['gridA']:
        for opt in ['423','855','387']:
            for hName in histStore[tag][opt]:
                if 'rate' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Lin. Option : '+opt
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['MarkerStyle']= 22
                pparams['LineSize'] = 2
                pparams['Options']='pec'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    i=1
    hist=baseline_rate
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='HIST pc'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

In [ ]:
if False:  # Rates GridC Best Options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='BestGridCPoints_rates',cPars=cPars) 
    plot.legendPosition = (0.55,0.60,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['Sigmoid Relaxation']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    hist=baseline_rate
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
#     i+=1
    for tag in ['gridC']:
        for opt in ['315','734','66','571']:
            for hName in histStore[tag][opt]:
                if 'rate' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Option : '+opt
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['MarkerStyle']= 22
                pparams['LineSize'] = 2
                pparams['Options']='pec'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

In [ ]:
histStore['gridA']['143']['rate_Progression143_5p0_0p9_45p0']

In [ ]:
if True:  # Rates GridA Best Options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='BestGridAPoints_rates',cPars=cPars) 
    plot.legendPosition = (0.55,0.50,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['Linear Relaxation']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (20.0,40.0) 
    plot.yRange = (5,70) 
    plot.logy = False
    plots.append(plot)
    
    i=1
    hist=baseline_rate
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
#     i+=1
    for tag in ['gridA']:
        for opt in ['153','159','716','589','454','159','438']:
            for hName in histStore[tag][opt]:
                if 'rate' not in hName:
                    continue
                i+=1
                print(hName)
                hist=histStore[tag][opt][hName].Clone()
                hist.Print()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Option : '+opt
                print(opt,col[i])
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['MarkerStyle']= 22
                pparams['LineSize'] = 2
                pparams['Options']='pec'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

In [ ]:
if False:  # Rates All Exotic Best option
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='ExoticPoints_rates',cPars=cPars) 
    plot.legendPosition = (0.55,0.55,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['Linear Relaxation']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    
    i+=1
    for tag in ['gridC']:
        for opt in ['380','218']:
            for hName in histStore[tag][opt]:
                if 'rate' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='#sigma-oid Option : '+opt
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['MarkerStyle']= 22
                pparams['LineSize'] = 2
                pparams['Options']='pec'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    for tag in ['gridA']:
        for opt in ['139','142','569']:
            for hName in histStore[tag][opt]:
                if 'rate' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Lin. Option : '+opt
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['MarkerStyle']= 22
                pparams['LineSize'] = 2
                pparams['Options']='pec'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    i=1
    hist=baseline_rate
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='HIST pc'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

## Efficiencies

In [ ]:
if False:  #  All Grids best options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='BestPoints_effficiency',cPars=cPars) 
    plot.legendPosition = (0.5,0.40,0.9,0.65)
    plot.descPosition   = (0.60,0.70)
    plot.desc =['']
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offline}"
    plot.xRange = (10.0,60.0) 
#         plot.yRange = (1e-3,yMax[hName]*1e1) 
    plot.yRange = (0.0,1.02) 
    plot.logy = False
    plots.append(plot)
    
    i=1
    hist=baseline_eff
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline , E_{T}^{L1} > 25 GeV'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
#     i+=1
    for tag in ['gridC']:
        for opt in ['734','66','571']:
            for hName in histStore[tag][opt]:
                if 'divide_pt_pass_' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='#sigma-oid Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[8]
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['LineSize'] = 2
                pparams['Options']='pce'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    for tag in ['gridA']:
        for opt in ['423','855','387','281']:
            for hName in histStore[tag][opt]:
                if 'divide_pt_pass_' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Lin. Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[8]
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['LineSize'] = 2
                pparams['Options']='pce'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

In [ ]:
if False:  #  GridB Best Options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='BestGridCPoints_effficiency',cPars=cPars) 
    plot.legendPosition = (0.55,0.40,0.9,0.55)
    plot.descPosition   = (0.60,0.70)
    plot.desc =['Sigmoid Relaxaton']
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offline}"
    plot.xRange = (10.0,60.0) 
#         plot.yRange = (1e-3,yMax[hName]*1e1) 
    plot.yRange = (0.0,1.02) 
    plot.logy = False
    plots.append(plot)
    
    i=1
    hist=baseline_eff
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline , E_{T}^{L1} > 25 GeV'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    for tag in ['gridC']:
        for opt in ['315','734','66','571']:
            for hName in histStore[tag][opt]:
                if 'divide_pt_pass_' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[8]
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['LineSize'] = 2
                pparams['Options']='pce'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

In [ ]:
if True:  #  GridA Best Options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='BestGridAPoints_effficiency',cPars=cPars) 
    plot.legendPosition = (0.55,0.30,0.9,0.58)
    plot.descPosition   = (0.60,0.70)
    plot.desc =['Linear Relaxation']
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offline}"
    plot.xRange = (5.0,80.0) 
    plot.yRange = (0.0,1.02) 
#     plot.xRange = (25.0,50.0) 
#     plot.yRange = (0.7,1.02) 
    plot.logy = False
    plots.append(plot)
    
    i=1
    hist=baseline_eff
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline , E_{T}^{L1} > 34 GeV'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)

    
#     i+=1
    for tag in ['gridA']:
        for opt in ['153','159','716','589','454','159','438']:
            for hName in histStore[tag][opt]:
                if 'divide_pT_pass_' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
#                 pparams['Legend']='Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[8]
                pparams['Legend']='Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[7].replace('Et','')
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['LineSize'] = 2
                pparams['Options']='pce'
                pparams['isTH1']=False
                print("Opt ",opt," col :",col[i])
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

In [ ]:
if False:  #  All Grids Exotic options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='ExoticPoints_effficiency',cPars=cPars) 
    plot.legendPosition = (0.5,0.40,0.9,0.65)
    plot.descPosition   = (0.60,0.70)
    plot.desc =['']
    plot.yTitle = "#epsilon"  
    plot.xTitle = "E_{T}^{offline}"
    plot.xRange = (10.0,60.0) 
#         plot.yRange = (1e-3,yMax[hName]*1e1) 
    plot.yRange = (0.0,1.02) 
    plot.logy = False
    plots.append(plot)
    
    i=1
    hist=baseline_eff
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline , E_{T}^{L1} > 25 GeV'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    for tag in ['gridC']:
        for opt in ['370','218']:
            for hName in histStore[tag][opt]:
                if 'divide_pt_pass_' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='#sigma-oid Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[8]
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['LineSize'] = 2
                pparams['Options']='pce'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    for tag in ['gridA']:
        for opt in ['139','142','569']:
            for hName in histStore[tag][opt]:
                if 'divide_pt_pass_' not in hName:
                    continue
                i+=1
                hist=histStore[tag][opt][hName].Clone()
                pparams=getDefaultPlotParams(col=i,marker=3)
                pparams['Legend']='Lin. Option : '+opt + " | E_{T}^{L1} > "+hName.split('_')[8]
                pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
                pparams['LineSize'] = 2
                pparams['Options']='pce'
                pparams['isTH1']=False
                aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
                aplot.normalize = False ;aplot.scaleFactor = None
        #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
                plots[-1].addPlot(aplot)
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

## Isolation Scans Pick And Plot

    For checking and identifying if the best turn ons are okay at low ets 

In [ ]:
'workarea/gridV3_122X/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridAp82.root'

In [ ]:
fStore={}
fStore['p82']=ROOT.TFile('workarea/gridV3_122X/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridCp82.root')
fStore['p64']=ROOT.TFile('workarea/gridV3_122X/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridCp64.root')

hStore={}
hStore['p82']=getTheObjectsFromFile(fStore['p82'])
hStore['p64']=getTheObjectsFromFile(fStore['p64'])

In [ ]:
histsToPlot={}
histsToPlot['Option 571']=[]
for hName in hStore['p64']['turn_on_progression'].keys():
    if '571' not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in [8,12,15,18]:
        continue
    print("Got ",hName)
    histsToPlot['Option 571'].append(hStore['p64']['turn_on_progression'][hName])
print()
histsToPlot['Option 734']=[]
for hName in hStore['p82']['turn_on_progression'].keys():
    if '734' not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in [8,12,15,18]:
        continue
    print("Got ",hName)
    histsToPlot['Option 734'].append(hStore['p82']['turn_on_progression'][hName])    

In [ ]:
if False:  #  Opt sig 734 Best Options
    plots=[]
    for tag in histsToPlot:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='EfffCkeckFor'+tag+'_effficiency',cPars=cPars) 
        plot.legendPosition = (0.55,0.40,0.9,0.58)
        plot.descPosition   = (0.60,0.70)
        plot.desc =['Sigmoid Relaxation',tag]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (4.0,40.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.02) 
        plot.logy = False
        plots.append(plot)

        i=1
        hist=baseline_eff
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Baseline , E_{T}^{L1} > 25 GeV'
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='pce'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1

        for hist in histsToPlot[tag]:
            print(hist)
            i+=1
            hName=hist.GetName()
            pparams=getDefaultPlotParams(col=i,marker=3)
            pparams['Legend']="E_{T}^{L1} > "+hName.split('_')[8]
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
            pparams['LineSize'] = 2
            pparams['Options']='pce'
            pparams['isTH1']=False
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor = None
    #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
            plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
fStore['p82rateCrosscheck']=ROOT.TFile('workarea/gridV3_122X/rateCrossCheck/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridC_120XRatesp82.root')
fStore['p64rateCrosscheck']=ROOT.TFile('workarea/gridV3_122X/rateCrossCheck/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridC_120XRatesp64.root')

hStore['p82rateCrosscheck']=getTheObjectsFromFile(fStore['p82rateCrosscheck'])
hStore['p64rateCrosscheck']=getTheObjectsFromFile(fStore['p64rateCrosscheck'])

histsToPlot['rate120X_734']=hStore['p82rateCrosscheck']['rate_progression']['rate_Progression734_22p5_1p0_6']
histsToPlot['rate120X_571']=hStore['p64rateCrosscheck']['rate_progression']['rate_Progression571_17p5_30_0p8']

In [ ]:
if False:  # Rates GridA Best Options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='RatesCrossCheck_734_571_',cPars=cPars) 
    plot.legendPosition = (0.55,0.60,0.9,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['Sigmoid Relaxation']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    hist=baseline_rate
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']='Baseline'
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['LineSize'] = 2
    pparams['Options']='pce'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    for tag in ['rate120X_734','rate120X_571']:
        i+=1
        hist=histsToPlot[tag].Clone()
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']=tag
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['MarkerStyle']= 22
        pparams['LineSize'] = 2
        pparams['Options']='pec'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
#             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
        plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            
 

### Linear relax 

In [ ]:
fStore={}
fStore['pl95']=ROOT.TFile('workarea/gridV3_122X/gridA_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridAp95.root')
fStore['pl47']=ROOT.TFile('workarea/gridV3_122X/gridA_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridAp47.root')

hStore={}
hStore['pl95']=getTheObjectsFromFile(fStore['pl95'])
hStore['pl47']=getTheObjectsFromFile(fStore['pl47'])

In [ ]:
histsToPlot={}
histsToPlot['Option l423']=[]
for hName in hStore['pl47']['turn_on_progression'].keys():
#     print(hName)
    if '423' not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in [15,18,22,24,25,28]:
        continue
    print("Got ",hName)
    histsToPlot['Option l423'].append(hStore['pl47']['turn_on_progression'][hName])
print()

histsToPlot['Option l855']=[]
for hName in hStore['pl95']['turn_on_progression'].keys():
    if '855' not in hName:
        continue
    eT=int(hName.split('_')[8])
    if eT not in [15,18,22,24,25,28]:
        continue
    print("Got ",hName)
    histsToPlot['Option l855'].append(hStore['pl95']['turn_on_progression'][hName])
print()



In [ ]:
if False:  #  Opt sig 734 Best Options
    plots=[]
    for tag in histsToPlot:
        cPars=getCanvasParams('Run3MC')
        plot=getDefaultPlot(prefix=pltPrefix,name='EfffCkeckFor'+tag+'_effficiency',cPars=cPars) 
        plot.legendPosition = (0.55,0.40,0.9,0.58)
        plot.descPosition   = (0.60,0.70)
        plot.desc =['Sigmoid Relaxation',tag]
        plot.yTitle = "#epsilon"  
        plot.xTitle = "E_{T}^{offline}"
        plot.xRange = (4.0,80.0) 
    #         plot.yRange = (1e-3,yMax[hName]*1e1) 
        plot.yRange = (0.0,1.02) 
        plot.logy = False
        plots.append(plot)

        i=1
        hist=baseline_eff
        pparams=getDefaultPlotParams(col=i,marker=3)
        pparams['Legend']='Baseline , E_{T}^{L1} > 25 GeV'
        pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
        pparams['LineSize'] = 2
        pparams['Options']='pce'
        pparams['isTH1']=False
        aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
        aplot.normalize = False ;aplot.scaleFactor = None
        plots[-1].addPlot(aplot)

    #     i+=1

        for hist in histsToPlot[tag]:
            print(hist)
            i+=1
            hName=hist.GetName()
            pparams=getDefaultPlotParams(col=i,marker=3)
            pparams['Legend']="E_{T}^{L1} > "+hName.split('_')[8]
            pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
            pparams['LineSize'] = 2
            pparams['Options']='pce'
            pparams['isTH1']=False
            aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
            aplot.normalize = False ;aplot.scaleFactor = None
    #             aplot.normalize = True ;aplot.scaleFactor = 1.0*xSections[tag]
            plots[-1].addPlot(aplot)

    canvas = []
    for plot in plots:
        canvas.append(plot.plot())             

In [ ]:
fStore['p82rateCrosscheck']=ROOT.TFile('workarea/gridV3_122X/rateCrossCheck/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridC_120XRatesp82.root')
fStore['p64rateCrosscheck']=ROOT.TFile('workarea/gridV3_122X/rateCrossCheck/gridC_v2/HistgramFile_step3step4_12XnewLayer1_Scan122XGridC_120XRatesp64.root')

hStore['p82rateCrosscheck']=getTheObjectsFromFile(fStore['p82rateCrosscheck'])
hStore['p64rateCrosscheck']=getTheObjectsFromFile(fStore['p64rateCrosscheck'])

histsToPlot['rate120X_734']=hStore['p82rateCrosscheck']['rate_progression']['rate_Progression734_22p5_1p0_6']
histsToPlot['rate120X_571']=hStore['p64rateCrosscheck']['rate_progression']['rate_Progression571_17p5_30_0p8']

In [ ]:
fileListForRatePFAp={
   'preHCAL'  : 'data/HistgramFile_Rate_Run3MC_Run3IsoLUT_PU44to52.root',
   'postHCAL' : 'data/HistgramFile_Rate_Run3MC_2022v0_2_recalibV1_withPFA1.root',
   'preHCALHighPU' : 'data/HistgramFile_Rate_Run3MC_2022v0_2_recalibV1_49To55.root',
}

histStoreForRatePFAp={}
fileStoreForRatePFAp={}
for tag in fileListForRatePFAp:
    print('Tag : ',tag)
    fileStoreForRatePFAp[tag]=ROOT.TFile(fileListForRatePFAp[tag],'READ')
    histStoreForRatePFAp[tag]=getTheObjectsFromFile(fileStoreForRatePFAp[tag])

In [ ]:
histStoreForRatePFAp[tag]

In [ ]:
if True:  # Rates GridA Best Options
    plots=[]
    cPars=getCanvasParams('Run3MC')
    plot=getDefaultPlot(prefix=pltPrefix,name='pfa1_rates',cPars=cPars) 
    plot.legendPosition = (0.55,0.60,0.93,0.75)
    plot.descPosition   = (0.60,0.80)
    plot.desc =['Linear Relaxation']
    plot.yTitle = "Rate  [ kHz ]"  
    plot.xTitle = "E_{T}^{L1}"
    plot.xRange = (1.0,60.0) 
    plot.yRange = (1e-1,3e4) 
    plot.logy = True
    plots.append(plot)
    
    i=1
    tag='preHCAL'
    hist=histStoreForRatePFAp[tag]['SingleEG_rate_TightIso'].Clone()
    hist.Print()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="Without PFA1'"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    i+=1
    tag='preHCALHighPU'
    hist=histStoreForRatePFAp[tag]['SingleEG_rate_TightIso'].Clone()
    hist.Print()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="With PFA1'"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    
    
    i+=1
#     tag='preHCALHighPU'
    hist=histStore['gridA']['143']['rate_Progression143_5p0_0p9_45p0']
    hist.Print()
    pparams=getDefaultPlotParams(col=i,marker=3)
    pparams['Legend']="With PFA1', Run3 Prelims"
    pparams['MarkerColor']= col[i]; pparams['LineColor'] = col[i];
    pparams['MarkerStyle']= 22
    pparams['LineSize'] = 2
    pparams['Options']='pec'
    pparams['isTH1']=False
    aplot = Plot(Name=hist.GetName(), Histo=hist,**pparams)
    aplot.normalize = False ;aplot.scaleFactor = None
    plots[-1].addPlot(aplot)
    
    canvas = []
    for plot in plots:
        canvas.append(plot.plot())            